In [1]:
# function myRelu(y::AbstractArray{T}, model::JuMP.Model) where {T}
#     N = length(y)
    
#     empty!(model)
#     set_optimizer_attribute(model, MOI.Silent(), true)
#     @variable(model, x[1:N] >= zero(T))
#     x = model[:x]
#     @objective(
#         model,
#         Min,
#         x'x -2x'y + y'y,
#     )
#     optimize!(model)

#     return value.(x)
# end

In [2]:
using Statistics
using DiffOpt
using Flux
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
using OSQP
using JuMP
using ChainRulesCore

In [3]:
## prepare data
imgs = Flux.Data.MNIST.images()
labels = Flux.Data.MNIST.labels();

# Preprocessing
X = hcat(float.(reshape.(imgs, :))...) #stack all the images
Y = onehotbatch(labels, 0:9); # just a common way to encode categorical variables

test_X = hcat(float.(reshape.(Flux.Data.MNIST.images(:test), :))...)
test_Y = onehotbatch(Flux.Data.MNIST.labels(:test), 0:9)

# float64 to float16, to save memory
X = convert(Array{Float16,2}, X) 
test_X = convert(Array{Float16,2}, test_X)

X = X[:, 1:1000]
Y = Y[:, 1:1000];

┌ Warning: Flux's datasets are deprecated, please use the package MLDatasets.jl
└ @ Flux.Data /home/pika/.julia/packages/Flux/6o4DQ/src/data/Data.jl:17
┌ Warning: Flux's datasets are deprecated, please use the package MLDatasets.jl
└ @ Flux.Data /home/pika/.julia/packages/Flux/6o4DQ/src/data/Data.jl:17
┌ Warning: Flux's datasets are deprecated, please use the package MLDatasets.jl
└ @ Flux.Data /home/pika/.julia/packages/Flux/6o4DQ/src/data/Data.jl:17
┌ Warning: Flux's datasets are deprecated, please use the package MLDatasets.jl
└ @ Flux.Data /home/pika/.julia/packages/Flux/6o4DQ/src/data/Data.jl:17


## Custom Relu

In [4]:
"""
    relu method for a Matrix
"""
function myRelu(y::AbstractMatrix{T}; model = Model(() -> diff_optimizer(OSQP.Optimizer))) where {T}
    x̂ = zero(y)
    
    # model init
    N = length(y[:, 1])
    empty!(model)
    set_optimizer_attribute(model, MOI.Silent(), true)
    @variable(model, x[1:N] >= zero(T))
    
    for i in 1:size(y)[2]
        @objective(
            model,
            Min,
            x'x -2x'y[:, i]
        )
        optimize!(model)
        x̂[:, i] = value.(x)
    end
    return x̂
end

myRelu

In [5]:
function ChainRulesCore.rrule(::typeof(myRelu), y::AbstractArray{T}; model = Model(() -> diff_optimizer(OSQP.Optimizer))) where {T}
    
    pv = myRelu(y, model=model) 
    
    function pullback_myRelu(dx)
        x = model[:x]
        dy = zero(dx)
        
        for i in 1:size(y)[2]
            MOI.set.(
                model,
                DiffOpt.BackwardIn{MOI.VariablePrimal}(), 
                x,
                dx[:, i]
            ) 

            DiffOpt.backward(model)  # find grad

            dy[:, i] = MOI.get.(
                model,
                DiffOpt.BackwardOut{DiffOpt.LinearObjective}(), 
                x, 
            )  # coeff of `x` in -2x'y
            dy[:, i] = -2 * dy[:, i]
        end
        
        return (NO_FIELDS, dy)
    end
    return pv, pullback_myRelu
end

## Define the NN

In [6]:
m = Chain(
    Dense(784, 64),
    myRelu,
    Dense(64, 10),
    softmax
)

Chain(Dense(784, 64), myRelu, Dense(64, 10), softmax)

In [7]:
loss(x, y) = crossentropy(m(x), y) 
opt = ADAM(); # popular stochastic gradient descent variant

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y)) # cute way to find average of correct guesses

dataset = repeated((X,Y), 20) # repeat the data set, very low accuracy on the orig dataset
evalcb = () -> @show(loss(X, Y)) # callback to show loss

#14 (generic function with 1 method)

In [8]:
Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 5)); #took me ~5 minutes to train on CPU

@show accuracy(X,Y)
@show accuracy(test_X, test_Y);

loss(X, Y) = 2.2714f0
loss(X, Y) = 2.076394f0
loss(X, Y) = 1.9149227f0
loss(X, Y) = 1.7748072f0
loss(X, Y) = 1.6523327f0
accuracy(X, Y) = 0.723


┌ Warning: The addition operator has been used on JuMP expressions a large number of times. This warning is safe to ignore but may indicate that model generation is slower than necessary. For performance reasons, you should not add expressions in a loop. Instead of x += y, use add_to_expression!(x,y) to modify x in place. If y is a single variable, you may also use add_to_expression!(x, coef, y) for x += coef*y.
└ @ JuMP /home/pika/.julia/packages/JuMP/MIPb6/src/JuMP.jl:1240


accuracy(test_X, test_Y) = 0.6635


###  accuracy 
- Dense(..., relu) ~ 0.875
- Dense(...), myRelu ~ 0.543